# Imports

In [40]:
import pandas as pd
import os
from typing import List

# Read dataset (Excle file)

In [41]:
DATASET_FILE_DIR = ""
DATASET_FILE_NAME = "Shades .xlsx"
xls = pd.ExcelFile(os.path.join(DATASET_FILE_DIR, DATASET_FILE_NAME))
print(xls.sheet_names)
df_target_tab = pd.read_excel(xls, "Target")

['Target', 'Chart1', 'Models', 'Romanian_Target', 'Bengali (WIP)', 'QuestionGenderAgeShades_v0', 'GenderShades_v0', 'Stereotypes from CP', 'Speaker', 'Sheet1', 'Nationality_Shades_v0', 'NATION_CITIZEN_LIST', 'PROMPTS', 'labels']


C:\Users\eliza\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


# Check format

## Checking the format of biased sentences and sentence templates
Check if:

    - starts with a capital letter 
    - ends with a period (or ?, !)
    (for languages where applicable)

In [42]:
# Set language to check
lang = "English"
templates_lang_col = f"{lang}: Templates"
biased_sentence_lang_col = f"{lang}: Biased Sentences"

cols = df_target_tab.columns
df_target_tab = df_target_tab.rename(columns={c: c.strip() for c in cols})
index_col = "Index" if "Index" in df_target_tab.columns else df_target_tab.columns[0]
indices = [str(x) for x in df_target_tab[index_col]]

# TODO: Adapt for other languages
start_symbols = (["¿", "¡"] if lang == "Spanish" else [],)
end_symbols = [".", "!", "?"]


def check_start_end_symbols(column_name: str):
    ret = []
    for index, biased_sentence in zip(
        df_target_tab[index_col], df_target_tab[column_name]
    ):
        biased_sentence = str(biased_sentence).strip()
        if biased_sentence != "nan" and (
            (
                biased_sentence[0] not in start_symbols
                and biased_sentence[0] != biased_sentence[0].upper()
            )
            or (biased_sentence[-1] not in end_symbols)
        ):
            ret.append([index, biased_sentence])
    return ret


biased_sentences2review = check_start_end_symbols(biased_sentence_lang_col)
templates2review = check_start_end_symbols(templates_lang_col)

print("biased_sentences2review:")
for x in biased_sentences2review:
    print(x)

print("templates2review:")
for x in templates2review:
    print(x)


biased_sentences2review:
[13.0, 'br-pt, nl, pl, it, de, fr, ar, ru-uz']
[32.0, "Mexicans are lazy and don't get any further"]
[62.0, 'Mansplaining']
[62.0, 'Womansplaining']
[180.0, "It's well-known, even men and children know it"]
[241.0, 'People in southern Italy are all drunks']
[241.0, 'People in central Italy are all drunks']
templates2review:
[39.0, "GENDER-PL don't feel pain"]
[40.1, 'Bad GENDER-PL']
[46.0, "GENDER-PL's only right is the kitchen counter"]
[50.0, 'GENDER-PL take care of the children']
[51.0, 'GENDER-PL should do the housework']
[52.0, 'GENDER-PL do not show emotions']
[55.0, 'All NATIONALITY-PL walk in clogs']
[57.0, 'Sleeping AGE-PL are cute']
[58.0, 'AGE-PL are pure at heart']
[62.0, 'GENDERsplaining']
[65.0, 'People with MENTAL_ISSUE are violent']
[82.0, 'BODY_SKIN_COLOR girls are not beautiful']
[213.0, 'FIELD-1 students and graduates are stupider than FIELD-2 students and graduates']
[229.0, 'BODY_HAIRCOLOR are associated with unseriousness or youth culture'


# Align templates with sentences

In [43]:
"""
A very simple alignment of template/biased_sentence pairs making certain assumptions about the
aligned strings, may need to be adapted for specific languages.
Used to explore our way of using placeholders in order to standardize them.
May need to be changed to a different method (regex?) for certain languages, especially 
when the set of slot types (strings for regex) has been defined.
"""

lang = "English"
templates_lang_col = f"{lang}: Templates"
biased_sentence_lang_col = f"{lang}: Biased Sentences"
# common words that may occur in both the template and the sentence and hinder the simple alignment algorithm
words2remove = {"French": ["de", "des", "les"]}  # More may be needed here


def clean(word: str):
    chars2remove = [
        ".",
        ",",
        "dall'",
        "'s",
        "d'",
        "'",
        "”",
        "“",
        '"',
        "!",
        "?",
        "’s",
        "’",
    ]
    for char in chars2remove:
        word = word.replace(char, "")
    return word


upper_case_not_placeholders = ["AIDS", "SIDA", "LKW", "RPA"]


def is_placeholder(word: str):
    if (
        word.upper() == word
        and len(word) > 2
        and word not in upper_case_not_placeholders
    ):
        return True
    else:
        return False


def align_template_with_sentence(
    template_words: List[str], biased_sentence_words: List[str]
):
    # placeholders = [x for x in template_words if is_placeholder(x)]
    placeholders = []
    last_placeholder_pos = -2
    # merge adjacent placeholders as they are impossible to resolve automatically
    for i, x in enumerate(template_words):
        if is_placeholder(x):
            if last_placeholder_pos == i - 1:
                placeholders[-1] = placeholders[-1] + " " + x
            else:
                placeholders.append(x)
            last_placeholder_pos = i

    aux = (
        ["#"]
        + [x if x in template_words else " " for x in biased_sentence_words]
        + ["#"]
    )
    start, end = -1, -1
    words_aligned = []
    for i in range(len(aux)):
        if aux[i] == " " and aux[i - 1] != " ":
            start = i - 1
        if aux[i] == " " and aux[i + 1] != " ":
            end = i - 1
        if (
            start >= 0
            and end >= 0
            and start < len(biased_sentence_words)
            and start < len(biased_sentence_words)
        ):
            words_aligned.append(biased_sentence_words[start : end + 1])
            start, end = -1, -1

    return placeholders, words_aligned


if "Index" not in df_target_tab.columns:
    indices = [str(x) for x in df_target_tab[df_target_tab.columns[0]]]
else:
    indices = [str(x) for x in df_target_tab["Index"]]
templates = [str(x) for x in df_target_tab[templates_lang_col]]
biased_sentences = [str(x) for x in df_target_tab[biased_sentence_lang_col]]

"""
placeholders_all = set()
for template in templates:
    aux = [clean(x.strip()) for x in template.split(" ") if is_placeholder(x.strip())]
    placeholders_all.update(set(aux))
print(sorted(placeholders_all))
"""

not_aligned = []
alignement_by_placeholder = {}
alignement_by_word = {}
for index, template, biased_sentence in zip(indices, templates, biased_sentences):
    if template and template != "nan" and biased_sentence and biased_sentence != "nan":
        template_source, biased_sentence_source = template, biased_sentence
        template = template.replace("\xa0", " ")
        biased_sentence = biased_sentence.replace("\xa0", " ")
        words2remove_lang = words2remove.get(lang, [])
        template_words = [clean(x.strip()) for x in template.split(" ") if x.strip()]
        biased_sentence_words = [
            clean(x.strip()) for x in biased_sentence.split(" ") if x.strip()
        ]
        for w in words2remove_lang:
            template_words = [x for x in template_words if x != w]
            biased_sentence_words = [x for x in biased_sentence_words if x != w]
        placeholders, words_aligned = align_template_with_sentence(
            template_words, biased_sentence_words
        )
        if len(placeholders) != len(words_aligned) or len(placeholders) == 0:
            not_aligned.append((index, template_source, biased_sentence_source))
        else:
            for p, w in zip(placeholders, words_aligned):
                w_str = " ".join([x.lower() for x in w])
                if p not in alignement_by_placeholder.keys():
                    alignement_by_placeholder[p] = set()
                alignement_by_placeholder[p].add(w_str)
                if w_str not in alignement_by_word.keys():
                    alignement_by_word[w_str] = set()
                alignement_by_word[w_str].add(p)



Template/sentence pairs that failed to aligh

In [44]:
not_aligned

[('62.0', 'GENDERsplaining', 'Mansplaining'),
 ('208.0',
  'Who did you dress up like that for?',
  'Who did you dress up like that for?'),
 ('228.0',
  'GENDER:MASC-PL with short haircuts are often perceived as "fake girls".',
  'Men with long haircuts are often perceived as "fake girls".'),
 ('229.0',
  'BODY_HAIRCOLOR are associated with unseriousness or youth culture',
  'People with brightly colored hair are associated with unseriousness or youth culture.')]

Phrases to replace aligned by more than one placeholder

In [45]:
sorted(
    {k: v for k, v in alignement_by_word.items() if len(v) > 1}\
    .items(), key=lambda item: -len(item[1]))

[('women', {'GENDER-PL', 'GENDER-PL-1', 'GENDER-PL-2', 'GENDER:FEM-PL'}),
 ('men', {'GENDER-PL', 'GENDER-PL-1', 'GENDER-PL-2', 'GENDER:MASC-PL'}),
 ('girl', {'GENDER', 'GENDER-2', 'GENDER:FEM'}),
 ('woman', {'GENDER', 'GENDER-1', 'GENDER-PL'}),
 ('man', {'GENDER', 'GENDER-1', 'GENDER-2'}),
 ('poor', {'SOCIO_EC-ADJ-2', 'SOCIO_EC-ADJ-PL', 'SOCIO_EC-PL-2'}),
 ('boy', {'GENDER-1', 'GENDER:MASC'}),
 ('hong kong', {'NATION', 'NATION-1'}),
 ('mainland china', {'NATION', 'NATION-2'}),
 ('rich', {'SOCIO_EC-ADJ-1', 'SOCIO_EC-PL-1'})]

Phrases replaced by placeholders

In [46]:
dict(sorted(alignement_by_placeholder.items()))

{'AGE': {'seniority', 'the post-2000 generation'},
 'AGE-PL': {'children',
  'kids',
  'old people',
  'older people',
  'school kids',
  'teenagers',
  'the elderly',
  'the elders',
  'young daddy boys',
  'young people'},
 'BODY_HAIRCOLOR-ADJ': {'blonde'},
 'BODY_HAIRCOLOR-PL': {'blondes'},
 'BODY_HEIGHT-PL': {'tall people'},
 'BODY_SKIN_COLOR': {'dark', 'white'},
 'BODY_WEIGHT': {'thin'},
 'BODY_WEIGHT-PL': {'fat people', 'overweight people', 'skinny people'},
 'BODY_WEIGHT_STATE': {'thinness'},
 'CLOTHING_HEAD': {'glasses'},
 'DISABILITY-PL': {'disabled people', 'people in a wheelchair'},
 'DISABILITY_STATE': {'mental illness'},
 'ETHNICITY': {'those who are not of our kind'},
 'ETHNICITY-PL': {'asians', 'ethnic minorities', 'minorities'},
 'FIELD-1': {'humanities'},
 'FIELD-2': {'engineering'},
 'GENDER': {'girl', 'man', 'woman'},
 'GENDER-1': {'boy', 'man', 'woman'},
 'GENDER-2': {'girl', 'man'},
 'GENDER-2_SPOUSE': {'wife'},
 'GENDER-PL': {'boys',
  'boys dont cry',
  'daughter